# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods

## Introduction

When a person is relocating to another city/neighbors due to various reasons like career development, family gathering and/or other personal reasons, it is of his/her preference to find the similar living evironment as previous live. Therefore, in this task, I will use the two cities (New York and Toronto)'s imformation to try to find out the similar neigbors/boroughs across cities or boroughs.

## 1. Data

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
import folium 
from bs4 import BeautifulSoup

In [16]:
# pip install foursquare

Note: you may need to restart the kernel to use updated packages.


### (1) Toronto Data

A) Extract Data from the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, which includes the neighbors, postcode and boroughs of Toronto

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

columns = ['PostalCode', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(columns) for key in columns})

for tr in table.find_all('tr'):
    for i,column in zip(tr.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

toronto_df = pd.DataFrame.from_dict(data=data)[columns]
print(toronto_df.shape)
toronto_df.head()

toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned'].reset_index(drop = True)
print(toronto_df.shape)
print(toronto_df[toronto_df['Neighbourhood'] == 'Not assigned'].shape[0])

p, b, n = [], [], []
for postcode, borough, neigh in zip(toronto_df['PostalCode'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    p.append(postcode)
    b.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)

toronto_df = pd.DataFrame({'PostalCode': p, 'Borough': b, 'Neighbourhood':n})[columns]
print(toronto_df.shape)
toronto_df.head()

postcodes = toronto_df['PostalCode'].values
boroughs = toronto_df['Borough'].values
neighs = toronto_df['Neighbourhood'].values

dic = dict({(key1,key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys in the dictionary are: ', len(dic.keys()))
dict

for postcode, borough, neigh in zip(postcodes,boroughs, neighs):
    key = (postcode, borough)
    dic[key].append(neigh)

toronto_df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])
for key, value in dic.items():
    postcode, borough, neig = key[0], key[1], value
    neig = ', '.join(neig)
    toronto_df = toronto_df.append({'Postal Code': postcode,
                     'Borough': borough,
                     'Neighbourhood': neig}, ignore_index = True)
print('Shape of final data is: ', toronto_df.shape)
toronto_df.head()

(180, 3)
(103, 3)
0
(103, 3)
Number of keys in the dictionary are:  103
Shape of final data is:  (103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


(2) Add the latitude and longitutde to the Toronto dataframe

In [5]:
try:
    toronto_df['Latitude'] = lats
    toronto_df['Longitude'] = lons
except:
    geo = pd.read_csv('Geospatial_Coordinates.csv')
    toronto_df = pd.merge(toronto_df, geo, how= 'inner', on = 'Postal Code')
    
print(toronto_df.shape)
toronto_df.head()

del toronto_df['Postal Code']

toronto_df.head()

(103, 5)


,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### (2) New York Data

In [6]:
with open('newyork_data.json') as f:
    ny_json = json.load(f)

ny_json = ny_json['features']
ny_json[0]

cols = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude']
ny_df = pd.DataFrame(columns = cols)
for data in ny_json:
    borough = data['properties']['borough']
    neigh   = data['properties']['name']
    lat_lon = data['geometry']['coordinates'] # now it'll return list
    lon, lat = lat_lon[0], lat_lon[1]
    
    ny_df = ny_df.append({'Borough': borough,'Neighbourhood': neigh, 'Latitude': lat,
                          'Longitude': lon}, ignore_index=True)
    
ny_df.head(10)

,Borough,Neighbourhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [5]:
# df = toronto_df.append(ny_df, ignore_index=True)
# print(df)

In [22]:
# ny_lats = ny_df['Latitude']
# ny_long = ny_df['Longitude']

In [20]:
# map_sv = folium.Map(location=[43.654260, -79.360636], zoom_start=12)

# # add markers to map
# for lat, lng, Borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
#     label = '{}, {}'.format(Borough, neighbourhood)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=2,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_sv)  
    
# map_sv

In [19]:
# map_sv = folium.Map(location=[40.894705,-73.847201], zoom_start=12)

# # add markers to map
# for lat, lng, Borough, neighbourhood in zip(ny_df['Latitude'], ny_df['Longitude'], ny_df['Borough'], ny_df['Neighbourhood']):
#     label = '{}, {}'.format(Borough, neighbourhood)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=2,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_sv)  
    
# map_sv

## 3.  Getting Venues Using Foursquare API

In [32]:
CLIENT_ID = 'CT0KIZNAXMABIZT234JBIWKPXGTTMBHMLN3ARJERQEOUS0CY' # your Foursquare ID
CLIENT_SECRET = '2J1N2RO2RKGVHIO23WKMFL51ZWRNKZN0JF2HFEVAUBABMYKP' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CT0KIZNAXMABIZT234JBIWKPXGTTMBHMLN3ARJERQEOUS0CY
CLIENT_SECRET:2J1N2RO2RKGVHIO23WKMFL51ZWRNKZN0JF2HFEVAUBABMYKP


In [35]:
radius = 1000
LIMIT = 200

venues = []
for lat, long, neighborhood, bor in zip(ny_df['Latitude'], ny_df['Longitude'], ny_df['Neighbourhood'], ny_df['Borough']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            bor,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [36]:
print(results)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4db73c8a5da3b5fa68a6628b', 'name': 'Roosevelt Island Promenade', 'location': {'address': 'Roosevelt island', 'lat': 40.76087483014197, 'lng': -73.95243668606233, 'labeledLatLngs': [{'label': 'display', 'lat': 40.76087483014197, 'lng': -73.95243668606233}], 'distance': 310, 'postalCode': '10044', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['Roosevelt island', 'New York, NY 10044', 'United States']}, 'categories': [{'id': '4d4b7105d754a06377d81259', 'name': 'Outdoors & Recreation', 'pluralName': 'Outdoors & Recreation', 'shortName': 'Outdoors & Recreation', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4db73c8a5da3b5fa68a6628b-0'}, {'reasons': {'count': 0

In [37]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Borough', 'Latitude','Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(8473, 8)


,Neighborhood,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Wakefield,Bronx,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,Bronx,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,Bronx,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,Bronx,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,Bronx,40.894705,-73.847201,Rite Aid,40.889062,-73.842993,Pharmacy


In [38]:
venues_df.head(15)

,Neighborhood,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Wakefield,Bronx,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,Bronx,40.894705,-73.847201,Ripe Kitchen & Bar,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,Bronx,40.894705,-73.847201,Ali's Roti Shop,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,Bronx,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,Bronx,40.894705,-73.847201,Rite Aid,40.889062,-73.842993,Pharmacy
5,Wakefield,Bronx,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
6,Wakefield,Bronx,40.894705,-73.847201,Jimbo's,40.891740,-73.858226,Burger Joint
7,Wakefield,Bronx,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
8,Wakefield,Bronx,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
9,Wakefield,Bronx,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy


In [70]:
df1 = venues_df.loc[venues_df['VenueCategory'].str.contains('Shop') == True]
df1['VenueCategory'].unique()

array(['Dessert Shop', 'Ice Cream Shop', 'Donut Shop',
       'Mobile Phone Shop', 'Food & Drink Shop', 'Flower Shop',
       'Shopping Mall', 'Bagel Shop', 'Coffee Shop', 'Miscellaneous Shop',
       'Sporting Goods Shop', 'Cosmetics Shop', 'Wine Shop',
       'Supplement Shop', 'Automotive Shop', 'Shop & Service',
       'Frozen Yogurt Shop', 'Gourmet Shop', 'Smoke Shop',
       'Fish & Chips Shop', 'Gift Shop', 'Hobby Shop', 'Cupcake Shop',
       'Print Shop', 'Cheese Shop', 'Optical Shop', 'Bubble Tea Shop',
       'Bike Shop', 'Record Shop', 'Other Repair Shop', 'Comic Shop',
       'Antique Shop', 'Pie Shop', 'Motorcycle Shop', 'Souvenir Shop',
       'Shopping Plaza', 'Board Shop', 'Tailor Shop', 'Chocolate Shop'],
      dtype=object)

In [71]:

# Sorting the rows by Venue Category
df1.sort_values(by=['VenueCategory'],inplace=True)
df1.head(10)

/Users/hchiawie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
6316,Boerum Hill,Brooklyn,40.685683,-73.983748,Collier West,40.687346,-73.985640,Antique Shop
6246,Downtown,Brooklyn,40.690844,-73.983463,Collier West,40.687346,-73.985640,Antique Shop
4879,Red Hook,Brooklyn,40.676253,-74.012759,Erie Basin,40.676834,-74.013822,Antique Shop
3992,Windsor Terrace,Brooklyn,40.656946,-73.980073,Windsor Place Antiques,40.658858,-73.978674,Antique Shop
2576,Castle Hill,Bronx,40.819014,-73.848027,Cummins Sales and Service,40.826142,-73.842652,Automotive Shop
2891,Edenwald,Bronx,40.884561,-73.848083,Firestone Complete Auto Care,40.877446,-73.844776,Automotive Shop
180,Eastchester,Bronx,40.887556,-73.827806,4 Star Auto Glass,40.885905,-73.827323,Automotive Shop
2634,Olinville,Bronx,40.871371,-73.863324,Valvoline Instant Oil Change,40.870969,-73.855866,Automotive Shop
5013,Gowanus,Brooklyn,40.673931,-73.994441,Bagel Pub Park Slope,40.669526,-73.986995,Bagel Shop
4956,Red Hook,Brooklyn,40.676253,-74.012759,Park Bagels,40.682975,-74.005456,Bagel Shop


In [72]:

# Creating Dummy Variables for Venue Category
onehot = pd.get_dummies(df1['VenueCategory'],prefix='',prefix_sep='')

In [73]:
df3=df1

onehot.head()
df3.head()
# Merging the dummy variables with the original dataframe
df4 = pd.merge(df3,onehot,on=df3.index)
df4.head(50)
df4.drop(['key_0'],axis=1,inplace=True)

In [74]:
group_clustre = df4.drop(['Neighborhood','Borough','Latitude','Longitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory'],axis=1)

group_clustre.head()

,Antique Shop,Automotive Shop,Bagel Shop,Bike Shop,Board Shop,Bubble Tea Shop,Cheese Shop,Chocolate Shop,Coffee Shop,Comic Shop,Cosmetics Shop,Cupcake Shop,Dessert Shop,Donut Shop,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Frozen Yogurt Shop,Gift Shop,Gourmet Shop,Hobby Shop,Ice Cream Shop,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Optical Shop,Other Repair Shop,Pie Shop,Print Shop,Record Shop,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Souvenir Shop,Sporting Goods Shop,Supplement Shop,Tailor Shop,Wine Shop
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
k=5
kmeans = KMeans(n_clusters=k,random_state=0).fit(group_clustre)
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 4, 4], dtype=int32)

In [76]:

kmeans.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [77]:
df4.head()
df4.insert(0,'ClusterLabels',kmeans.labels_)

In [89]:
# create map

map_clusters = folium.Map(location=[40.685683,-73.983748], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, name, cat, cluster in zip(df4['VenueLatitude'], df4['VenueLongitude'], df4['VenueName'],df4['VenueCategory'], df4['ClusterLabels']):
    label = folium.Popup(name +','+ cat + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
# for lat, lng, bor, neighbourhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
#     label = '{}, {}'.format(bor, neighbourhood)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_clusters) 

    

    
map_clusters

In [96]:
cluster0 = df4.loc[df4['ClusterLabels'] == 0, df4.columns[[1] + list(range(5, df4.shape[1]))]]
cluster1 = df4.loc[df4['ClusterLabels'] == 1, df4.columns[[1] + list(range(5, df4.shape[1]))]]
cluster2 = df4.loc[df4['ClusterLabels'] == 2, df4.columns[[1] + list(range(5, df4.shape[1]))]]
cluster3 = df4.loc[df4['ClusterLabels'] == 3, df4.columns[[1] + list(range(5, df4.shape[1]))]]
cluster4 = df4.loc[df4['ClusterLabels'] == 4, df4.columns[[1] + list(range(5, df4.shape[1]))]]

In [98]:
print(cluster0['VenueCategory'].value_counts())
print(cluster1['VenueCategory'].value_counts())
print(cluster2['VenueCategory'].value_counts())
print(cluster3['VenueCategory'].value_counts())
print(cluster4['VenueCategory'].value_counts())

Mobile Phone Shop      96
Wine Shop              78
Cosmetics Shop         57
Dessert Shop           39
Gourmet Shop           36
Supplement Shop        27
Food & Drink Shop      27
Gift Shop              24
Shopping Mall          19
Sporting Goods Shop    19
Miscellaneous Shop     16
Frozen Yogurt Shop     16
Flower Shop            14
Optical Shop           14
Record Shop            12
Smoke Shop             11
Bubble Tea Shop        10
Cupcake Shop            7
Cheese Shop             7
Fish & Chips Shop       6
Pie Shop                5
Bike Shop               5
Antique Shop            4
Automotive Shop         4
Board Shop              3
Hobby Shop              2
Comic Shop              2
Other Repair Shop       2
Motorcycle Shop         2
Shop & Service          2
Shopping Plaza          1
Print Shop              1
Chocolate Shop          1
Souvenir Shop           1
Tailor Shop             1
Name: VenueCategory, dtype: int64
Coffee Shop    252
Name: VenueCategory, dtype: int64
Don

In [99]:
radius = 1000
LIMIT = 200

venues = []
for lat, long, neighborhood, bor in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood'], toronto_df['Borough']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            bor,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

KeyError: 'groups'

In [100]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Borough', 'Latitude','Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(316, 8)


,Neighborhood,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Parkwoods,North York,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,North York,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,North York,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,Parkwoods,North York,43.753259,-79.329656,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop


In [101]:
df1_t = venues_df.loc[venues_df['VenueCategory'].str.contains('Shop') == True]
df1_t['VenueCategory'].unique()

array(['Fish & Chips Shop', 'Food & Drink Shop', 'Coffee Shop',
       'Shopping Mall', 'Cosmetics Shop', 'Shop & Service',
       'Chocolate Shop', 'Dessert Shop', 'Ice Cream Shop', 'Cheese Shop',
       'Miscellaneous Shop', 'Bubble Tea Shop', 'Hobby Shop',
       'Smoke Shop', 'Gift Shop', 'Gourmet Shop', 'Comic Shop'],
      dtype=object)

In [102]:

# Sorting the rows by Venue Category
df1_t.sort_values(by=['VenueCategory'],inplace=True)
df1_t.head(10)

/Users/hchiawie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighborhood,Borough,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
260,"Queen's Park, Ontario Provincial Government",Downtown Toronto,43.662301,-79.389494,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
190,"Queen's Park, Ontario Provincial Government",Downtown Toronto,43.662301,-79.389494,The Alley,43.665922,-79.385567,Bubble Tea Shop
163,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763,Grande Cheese,43.720146,-79.457951,Cheese Shop
56,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
160,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763,Tim Hortons,43.719427,-79.467995,Coffee Shop
112,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,The Cannonball,43.658299,-79.352397,Coffee Shop
130,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,cafe 260,43.653183,-79.370115,Coffee Shop
136,"Regent Park, Harbourfront",Downtown Toronto,43.654260,-79.360636,Savoury Grounds,43.656821,-79.358970,Coffee Shop
306,"Malvern, Rouge",Scarborough,43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
167,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763,Tim Hortons,43.722954,-79.456065,Coffee Shop


In [115]:
# Creating Dummy Variables for Venue Category
onehot_t = pd.get_dummies(df1_t['VenueCategory'],prefix='',prefix_sep='')

In [116]:
df3_t=df1_t

onehot_t.head()
df3_t.head()
# Merging the dummy variables with the original dataframe
df4_t = pd.merge(df3_t,onehot_t,on=df3_t.index)
df4_t.head(50)
df4_t.drop(['key_0'],axis=1,inplace=True)

In [117]:
group_cluster = df4_t.drop(['Neighborhood','Borough','Latitude','Longitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory'],axis=1)

group_cluster.head()

,Bubble Tea Shop,Cheese Shop,Chocolate Shop,Coffee Shop,Comic Shop,Cosmetics Shop,Dessert Shop,Fish & Chips Shop,Food & Drink Shop,Gift Shop,Gourmet Shop,Hobby Shop,Ice Cream Shop,Miscellaneous Shop,Shop & Service,Shopping Mall,Smoke Shop
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [118]:
k=5
kmeans = KMeans(n_clusters=k,random_state=0).fit(group_cluster)
kmeans.labels_[0:10]


array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [119]:
kmeans.labels_

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 3, 3, 3, 3, 0], dtype=int32)

In [120]:
df4_t.head()
df4_t.insert(0,'ClusterLabels',kmeans.labels_)

In [121]:
# create map

map_clusters = folium.Map(location=[40.685683,-73.983748], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, name, cat, cluster in zip(df4['VenueLatitude'], df4['VenueLongitude'], df4['VenueName'],df4['VenueCategory'], df4['ClusterLabels']):
    label = folium.Popup(name +','+ cat + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [124]:
cluster0 = df4_t.loc[df4_t['ClusterLabels'] == 0, df4_t.columns[[1] + list(range(5, df4_t.shape[1]))]]
cluster1 = df4_t.loc[df4_t['ClusterLabels'] == 1, df4_t.columns[[1] + list(range(5, df4_t.shape[1]))]]
cluster2 = df4_t.loc[df4_t['ClusterLabels'] == 2, df4_t.columns[[1] + list(range(5, df4_t.shape[1]))]]
cluster3 = df4_t.loc[df4_t['ClusterLabels'] == 3, df4_t.columns[[1] + list(range(5, df4_t.shape[1]))]]
cluster4 = df4_t.loc[df4_t['ClusterLabels'] == 4, df4_t.columns[[1] + list(range(5, df4_t.shape[1]))]]

In [125]:
print(cluster0['VenueCategory'].value_counts())
print(cluster1['VenueCategory'].value_counts())
print(cluster2['VenueCategory'].value_counts())
print(cluster3['VenueCategory'].value_counts())
print(cluster4['VenueCategory'].value_counts())

Dessert Shop          3
Miscellaneous Shop    2
Bubble Tea Shop       2
Comic Shop            1
Cheese Shop           1
Hobby Shop            1
Gift Shop             1
Food & Drink Shop     1
Shop & Service        1
Fish & Chips Shop     1
Chocolate Shop        1
Gourmet Shop          1
Smoke Shop            1
Name: VenueCategory, dtype: int64
Coffee Shop    31
Name: VenueCategory, dtype: int64
Cosmetics Shop    4
Name: VenueCategory, dtype: int64
Shopping Mall    4
Name: VenueCategory, dtype: int64
Ice Cream Shop    3
Name: VenueCategory, dtype: int64
